In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
(X_data, y_data), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print (X_data.shape)

# reduce the dimention of X_train, X_test
def reduce_D (data):
    return data.astype (np.float32).reshape (-1, 28 * 28) / 255.0
X_data, X_test = reduce_D  (X_data), reduce_D (X_test)
print (X_data.shape, X_test.shape)

y_data = y_data.astype(np.int32)
y_test = y_test.astype(np.int32)


X_data_trimmed = X_data[y_data < 5]
y_data_trimmed = y_data [y_data < 5]

X_test = X_test [y_test < 5]
y_test = y_test [y_test < 5]

print (X_data_trimmed.shape, X_test.shape, y_data_trimmed.shape, y_test.shape)

# Check Y :
# print (y_data.shape)
# def y_preprocess (data):
#     return data.astype (np.int32).reshape (data.shape[0], 1)
# y_data, y_test = y_preprocess (y_data), y_preprocess (y_test)
print (y_data.shape, y_test.shape)

# split the train and validation data:
X_validation, y_validation = X_data_trimmed[:5000], y_data_trimmed[:5000]
X_train, y_train = X_data_trimmed[5000:], y_data_trimmed[5000:]

print (X_validation.shape, X_train.shape, y_validation.shape, y_train.shape, X_test.shape, y_test.shape)

(60000, 28, 28)
(60000, 784) (10000, 784)
(30596, 784) (5139, 784) (30596,) (5139,)
(60000,) (5139,)
(5000, 784) (25596, 784) (5000,) (25596,) (5139, 784) (5139,)


In [16]:
reset_graph()
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 5

In [17]:
X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

In [18]:
with tf.name_scope ('DNN_chapter11_86'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    hidden1 = tf.layers.dense (X, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden1')
    hidden2 = tf.layers.dense (hidden1, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden2')
    hidden3 = tf.layers.dense (hidden2, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden3')
    hidden4 = tf.layers.dense (hidden3, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden4')
    hidden5 = tf.layers.dense (hidden4, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden5')
    y_predit = tf.layers.dense (hidden5, n_output, kernel_initializer = he_init, name = 'Output')
    y_proba = tf.nn.softmax (y_predit, name = "y_predit")

In [19]:
with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

In [20]:
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [21]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [22]:
init = tf.global_variables_initializer()
saver = tf.train.Saver ()
graph_
save_path = r"C:\Users\asiaynrf\Desktop\fun\handson_ML\model_saver"
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [25]:
import time
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        validation_score = accuracy.eval (feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
        saver.save (sess, save_path + '/curr_model.ckpt')
    total_end_time = time.time ()
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  1.0 the validation score is  0.9646 this cost 2.168649435043335
At step  1 the train_score is  1.0 the validation score is  0.9722 this cost 2.1577701568603516
At step  2 the train_score is  1.0 the validation score is  0.9758 this cost 2.3181517124176025
At step  3 the train_score is  1.0 the validation score is  0.9778 this cost 2.2239668369293213
At step  4 the train_score is  1.0 the validation score is  0.9788 this cost 2.273341178894043
At step  5 the train_score is  1.0 the validation score is  0.9794 this cost 2.2438929080963135
At step  6 the train_score is  1.0 the validation score is  0.9804 this cost 2.20210862159729
At step  7 the train_score is  1.0 the validation score is  0.9814 this cost 2.241654396057129
At step  8 the train_score is  1.0 the validation score is  0.9816 this cost 2.2515370845794678
At step  9 the train_score is  1.0 the validation score is  0.9826 this cost 2.2176175117492676
At step  10 the train_score is  1.0 the valid

In [27]:
with tf.Session () as sess: 
    saver.restore (sess, save_path + '/curr_model.ckpt')
    accuracy_test = accuracy.eval (feed_dict = {X: X_test, y: y_test})

INFO:tensorflow:Restoring parameters from C:\Users\asiaynrf\Desktop\fun\handson_ML\model_saver/curr_model.ckpt


In [28]:
accuracy_test

0.99046504